In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 
import numpy as np

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.


In [3]:
root2 = math.sqrt(2)
root3 = math.sqrt(3)

invroot2 = 1/root2 

# matrix representation of unitary operators

z_mat = [ [ 1 , 0 ],
          [ 0 , -1] ]

x_mat = [ [ 0 , 1 ],
          [ 1 , 0] ]

w_mat = [ [ 0 , 0 ],
          [ 0 , 0] ]

v_mat = [ [ 0 , 0 ],
          [ 0 , 0] ]

v_third =[ [1/root3, root2/root3],
            [root2/root3, -1/root3]]
    

for i in range(2):
    for j in range(2):
        w_mat[i][j] = invroot2 * (x_mat[i][j] + z_mat[i][j])
        v_mat[i][j] = invroot2 * (x_mat[i][j] - z_mat[i][j])
        
print(v_mat)

[[-0.7071067811865475, 0.7071067811865475], [0.7071067811865475, 0.7071067811865475]]


In [4]:
# create qubit pair in desired state
def entangledPair():
    q = QuantumCircuit(2) 
    q.h(0)
    q.cx(0,1)
    return q

# define Alice and Bob's actions
def a1(q):
    q.unitary(x_mat, [1])
    return q

def a2(q):
    q.unitary(w_mat, [1])
    return q

def a3(q):
    q.unitary(z_mat, [1])
    return q

def b1(q):
    q.unitary(w_mat, [0])
    return q

def b2(q):
    q.unitary(z_mat, [0])
    return q

def b3(q):
    q.unitary(v_mat, [0])
    return q

alice_actions = [a1, a2, a3]
bob_actions = [b1, b2, b3]

alice_sequence = []
bob_sequence = []

In [47]:
# selects random action
def rand_choice():
    while True:
        circuit = QuantumCircuit(2)
        circuit.h(0)
        circuit.h(1)
        meas = QuantumCircuit(2, 2)
        meas.measure(range(2), range(2))
        qc = meas.compose(circuit, range(2), front=True)
        backend = BasicSimulator()
        qc_compiled = transpile(qc, backend)
        job_sim = backend.run(qc_compiled, shots=1)
        result_sim = job_sim.result()
        counts = result_sim.get_counts(qc_compiled)
        if '00' in counts:
            return 1
        elif '01' in counts:
            return 2
        elif '10' in counts:
            return 3


In [42]:
# Note: attempted to use a unitary operator for the given state 
# but could not figure out how to get equal outputs from that
# so used other quantum method instead :(

# def rand_choice():

#     v_third = [
#         [1/root3, root2/root3],
#         [root2/root3, -1/root3]]
    
#     # Create a quantum circuit with 2 qubits
#     circuit = QuantumCircuit(2)
    
#     # Apply the unitary gate to the first qubit
#     gate = UnitaryGate(v_third)
#     circuit.append(gate, [0])
#     circuit.h([0, 1])
#     circuit.measure_all()
    
#     backend = BasicSimulator()
#     compiled = transpile(circuit, backend)
#     job_sim = backend.run(compiled, shots=1)
#     result_sim = job_sim.result()
#     counts = result_sim.get_counts(compiled)
#     if '11' in counts or '01' in counts:
#         return 3
#     elif '10' in counts:
#         return 2
#     elif '00' in counts:
#         return 1


In [44]:
# generates key from opposite values
def seq_calc(list):
    seq = ""
    for v in list:
        backend = BasicSimulator()
        compiled = transpile(v, backend)
        job_sim = backend.run(compiled, shots=1)
        result_sim = job_sim.result() 
        counts = result_sim.get_counts(compiled)
        for key in counts:
            seq = seq + str(abs(int(key[0])))
    return seq

In [45]:
def average(c,n):
    c.measure_all()
    
    backend = BasicSimulator()
    compiled = transpile(c, backend)
    job_sim = backend.run(compiled, shots=n)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    count00 = counts.get("00",0) 
    count01 = counts.get("01",0) 
    count10 = counts.get("10",0) 
    count11 = counts.get("11",0)
    return (count00 - count01 - count10 + count11) / n

In [48]:
N = 100

xw_count = 0
xv_count = 0
zw_count = 0
zv_count = 0

opposite_results = []

#cases: for entangement check
circuit_A1_B1 = b1(a1(entangledPair()))
circuit_A1_B3 = b3(a1(entangledPair()))
circuit_A3_B1 = b1(a3(entangledPair()))
circuit_A3_B3 = b3(a3(entangledPair()))

for reps in range(int(9 * (N / 2))):

    i = rand_choice()
    j = rand_choice()
    
    # case i = 2, j = 1: opposite
    if i == 2 and j == 1:
        circuit_A2_B1 = b1(a2(entangledPair()))
        circuit_A2_B1.measure_all()
        opposite_results.append(circuit_A2_B1)
    
    # case i = 3, j = 2: opposite
    elif i == 3 and j == 2:
        circuit_A3_B2 = b2(a3(entangledPair()))
        circuit_A3_B2.measure_all()
        opposite_results.append(circuit_A3_B2)
        
    
    # case i = 1, j = 1: xw
    elif i == 1 and j == 1:
        xw_count += 1
    
    #case i = 1, j = 3: xv
    elif i == 1 and j == 3:
        xv_count += 1
    
    #case i = 3, j = 1: zw
    elif i == 3 and j == 1:
        zw_count += 1
    
    #case i = 3, j = 3: zv
    elif i == 3 and j == 3:
        zv_count += 1

shared_key = seq_calc(opposite_results)

xw_avg = average(circuit_A1_B1, xw_count)
xv_avg = average(circuit_A1_B3, xv_count)
zw_avg = average(circuit_A3_B1, zw_count)
zv_avg = average(circuit_A3_B3, zv_count)
s = abs(xw_avg - xv_avg + zw_avg + zv_avg)

print(shared_key)
print(s)
print(s - (2 * root2))

100111011001111110110000011001011010101101001111000010111100000111011110101101000011110110
0.14859857656467826
-2.679828548181512
